# Network Analysis

This notebook performs network analysis across the entire episode.

In [ ]:
# Import stuff
from pandas import DataFrame, read_csv
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber
from nipype.interfaces.fsl import GLM, Split, Merge, Cluster
from nipype.interfaces.fsl.maths import MathsCommand

# FSL set up- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# Set study variables
analysis_home = '/data/perlman/moochie/user_data/CamachoCat/ChEC/fmri_proc'
templates_dir = '/data/perlman/moochie/user_data/CamachoCat/Aggregate_anats/templates'
raw_dir = analysis_home + '/raw'
preproc_dir = analysis_home + '/preproc'
firstlevel_dir = analysis_home + '/subjectlevel'
network_dir = analysis_home + '/network_analysis'
workflow_dir = analysis_home + '/workflows'
affect_ratings = analysis_home + '/misc/ratings_20200427.csv'

gm_mask = templates_dir + '/lcbd_template_2mm_gm.nii.gz'

subject_info = read_csv(analysis_home + '/misc/subjectinfo.csv', index_col=None)
subjects_list = subject_info['SubjID'].tolist()

# data collection specs
TR = 0.8 #in seconds
duration= 1284

In [ ]:
# Select subjects list
infosource = Node(IdentityInterface(fields=['subjid']),
                  name='infosource')
infosource.iterables = [('subjid', subjects_list)]

# Pull files
file_template = {'preproc_func': preproc_dir + '/fully_processed_func/{subjid}/lomo_func.nii.gz'}
selectfiles = Node(SelectFiles(file_template), name='selectfiles')

# Sink data of interest 
substitutions = [('_subjid_', '')] #output file name substitutions
datasink = Node(DataSink(base_directory = network_dir,
                        container = network_dir,
                        substitutions = substitutions), 
                name='datasink')